In [11]:
#############################################################################
#          __________                                                       #
#   __  __/ ____/ __ \__ __   This file is part of byron v1.0 "Kiwi"     #
#  / / / / / __/ /_/ / // /   (!) by Giovanni Squillero and Alberto Tonda   #
# / /_/ / /_/ / ____/ // /_   https://github.com/cad-polito-it/byron         #
# \__  /\____/_/   /__  __/                                                 #
#   /_/ --byron-- /_/      "You don't need a big goal, be μ-ambitious!!" #
#                                                                           #
#############################################################################

In [19]:
import pickle
from abc import ABC, abstractmethod
from functools import wraps, cache

In [21]:
class FitnessABC(ABC):
    """Fitness of a phenotype, handle multiple formats (eg. scalar, tuple).

    The class also redefines the relational operator in order to handle different types of optimization
    (eg. maximization, minimization) and to provide limited support to more complex scenarios
    (eg. multi-objective optimization)

    Equalities ('==' and '!=') are based on `is_distinguishable`.

    Single angular-bracket operators ('>', '<', '>=', and '<=') are based on `is_fitter` and may be randomized
    (ie. the result may not be reproducible).

    Double angular-bracket operators ('>>' and '<<') are based on `is_dominant` and the result is stable. By default,
    `is_dominant` is defined as `is_fitter`.

    When subclassing, one should only redefine `is_fitter`, and optionally `is_distinguishable` and `is_dominant`;
    `is_dominant` **must** be changed if `is_fitter` is randomized, making the result not reproducible.

    Additional sanity checks should be added to `check_comparable
    `. Subclasses may redefine the `decorate` method to
    change the value appearance.
    """

    @abstractmethod
    def is_fitter(self, other: "FitnessABC") -> bool:
        """Check whether fitter than the other (result may be accidental)."""
        assert self.check_comparable(other)
        return super().__gt__(other)

    def is_dominant(self, other: "FitnessABC") -> bool:
        """Check whether dominates the other (result is certain)."""
        return self.is_fitter(other)

    def is_distinguishable(self, other: "FitnessABC") -> bool:
        """Check whether some differences from the other Fitness may be perceived."""
        assert self.check_comparable(other)
        return super().__ne__(other)

    def check_comparable(self, other: "FitnessABC"):
        assert str(self.__class__) == str(
            other.__class__
        ), f"TypeError: different Fitness types: {self.__class__} and {other.__class__} (paranoia check)"
        return True

    def _decorate(self) -> str:
        """Represent the individual fitness value with a nice string."""
        return f"{super().__str__()}"

    # FINAL/WARNINGS

    def __eq__(self, other) -> bool:
        return not self.is_distinguishable(other)

    def __ne__(self, other) -> bool:
        return self.is_distinguishable(other)

    def __gt__(self, other) -> bool:
        return self.is_fitter(other)

    def __lt__(self, other) -> bool:
        return other.is_fitter(self)

    def __ge__(self, other) -> bool:
        return not self.__lt__(other)

    def __le__(self, other) -> bool:
        return not self.__gt__(other)

    def __rshift__(self, other) -> bool:
        return self.is_dominant(other)

    def __lshift__(self, other) -> bool:
        return other.is_dominant(self)

    def __str__(self):
        # Double parentheses: ⸨ ⸩  (U+2E28, U+2E29)
        # White parentheses: ⦅ ⦆  (U+2985, U+2986)
        # Fullwidth white parentheses:｟ ｠ (U+FF5F, U+FF60)
        # Math white square parentheses: ⟦ ⟧ (U+27E6, U+27E7)
        # Z notation binding bracket: ⦉ ⦊
        # Curved angled bracket: ⧼ ⧽
        return self._decorate() + "Ƒ"

    def __repr__(self):
        return f"<{self.__class__.__module__}.{self.__class__.__name__} @ {hex(id(self))}>"

    def __hash__(self) -> int:
        return super().__hash__()

    def run_paranoia_checks(self) -> bool:
        return super().run_paranoia_checks()

In [22]:
class Integer(FitnessABC, int):
    """A single numeric value -- Larger is better."""

    def __new__(cls, *args, **kw):
        return int.__new__(cls, *args, **kw)

    def _decorate(self):
        return str(int(self))

In [23]:
a = Integer(42)
pickle.dumps(a)

b'\x80\x04\x95\x1e\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x07Integer\x94\x93\x94K*\x85\x94\x81\x94.'